In [2]:


# Start a Dask Client with 8 workers, each using a single thread (and thus a single CPU)
client = Client(n_workers=4, threads_per_worker=1)
print("Dashboard link:", client.dashboard_link)


Dashboard link: http://127.0.0.1:8787/status


In [50]:
# Setup the diagnostics tools
pbar = ProgressBar()
pbar.register()

def profile(func):
    """Profile the function using the Dask Profiler, ResourceProfiler, and CacheProfiler

    Args:
        func (function): The function to profile
    """
    def wrapper(*args, **kwargs):
        with Profiler() as prof, ResourceProfiler(dt=0.25) as rprof, CacheProfiler() as cprof:
            result = func(*args, **kwargs)
        print(prof.results)
        print(rprof.results)
        print(cprof.results)
        return result, prof, rprof, cprof
    return wrapper


In [34]:
train = dd.read_csv("train_FE.csv", blocksize='64MB')
train=train.drop('Unnamed: 0', axis=1)
X = train[train.columns[3:]]
X = X.drop(["pickup_day_of_week", "euc_distance"], axis=1)
y = train[train.columns[1:2]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

/Users/rohithreddykota/miniconda3/lib/python3.11/site-packages/dask_ml/model_selection/_split.py:464: FutureWarning: The default value for 'shuffle' must be specified when splitting DataFrames. In the future DataFrames will automatically be shuffled within blocks prior to splitting. Specify 'shuffle=True' to adopt the future behavior now, or 'shuffle=False' to retain the previous behavior.
  warnings.warn(


In [ ]:
# from dask.distributed import Client
import dask.dataframe as dd
from dask_ml.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, cross_val_predict
from dask_ml.linear_model import LinearRegression
from dask_ml.metrics import mean_squared_error, r2_score
import numpy as np

# Start a Dask client
# client = Client()

# Load the dataset using Dask
train = dd.read_csv("train_FE.csv")
train = train.drop("Unnamed: 0", axis=1)

# Preprocessing with Dask
X = train[train.columns[3:]]
X = X.drop(["pickup_day_of_week", "pickup_date", "euc_distance"], axis=1)
y = train[train.columns[1:2]]

# Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# Ensuring computations are triggered for shape
X_train_shape = X_train.shape.compute()
y_train_shape = y_train.shape.compute()
X_test_shape = X_test.shape.compute()
y_test_shape = y_test.shape.compute()

print(X_train_shape, y_train_shape)
print(X_test_shape, y_test_shape)

###################
# Linear regression
###################
lr = LinearRegression()
lr.fit(X_train, y_train)

# Cross-validation scores
scores = cross_val_score(lr, X_train, y_train, cv=6).compute()

# Making predictions
lr_y_pred = cross_val_predict(lr, X_test, y_test, cv=6).compute()

# Coefficients
# Note: Dask's LinearRegression doesn't provide coef_ directly like sklearn, might need to adjust based on the Dask ML version or model specifics.

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test, lr_y_pred))

# Calculate Mean Squared Error
mse = mean_squared_error(y_test, lr_y_pred)

# Calculate Variance Score (R^2)
variance_score = r2_score(y_test, lr_y_pred)

# Computing metrics (Ensure to trigger computation if they're dask objects)
print(f"RMSE: {rmse.compute()}")
print(f"Mean squared error: {mse.compute():.2f}")
print(f"Variance score: {variance_score.compute():.2f}")


In [58]:
import dask.dataframe as dd

# Load your dataset
df = dd.read_csv("train_FE.csv", blocksize='64MB')

# Preprocessing steps
df = df.drop("Unnamed: 0", axis=1)
X = df.drop(['fare_amount'], axis=1)
y = df['fare_amount']


In [57]:

# Assuming you need to convert to Dask Arrays for sklearn compatibility
import dask.array as da

X_dask = da.from_array(X, chunks=(10000, X.shape[1]))  # Adjust chunk sizes based on your dataset and available memory
y_dask = da.from_array(y, chunks=10000)

X_computed = X_dask.compute()  # Converts Dask Array to NumPy array
y_computed = y_dask.compute()  # Converts Dask Array to NumPy array


/Users/rohithreddykota/miniconda3/lib/python3.11/site-packages/dask/array/core.py:3469: UserWarning: Passing an object to dask.array.from_array which is already a Dask collection. This can lead to unexpected behavior.
  warnings.warn(


AttributeError: 'DataFrame' object has no attribute 'dtype'

In [ ]:

from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression

# Initialize the model
lr = LinearRegression()

# Perform cross-validation
# Note: Ensure your data size is manageable to fit into memory for this step
scores = cross_val_score(lr, X_computed, y_computed, cv=6)

print("Cross-Validation Scores:", scores)
print("Average Score:", scores.mean())


In [43]:
# from dask_ml.model_selection import train_test_split, cross_val_score, cross_val_predict
import dask_ml.model_selection.train_test_split


ModuleNotFoundError: No module named 'dask_ml.model_selection.train_test_split'

/Users/rohithreddykota/miniconda3/lib/python3.11/site-packages/dask_ml/model_selection/_split.py:464: FutureWarning: The default value for 'shuffle' must be specified when splitting DataFrames. In the future DataFrames will automatically be shuffled within blocks prior to splitting. Specify 'shuffle=True' to adopt the future behavior now, or 'shuffle=False' to retain the previous behavior.
  warnings.warn(


In [28]:
from dask_ml.linear_model import LinearRegression
from dask_ml.model_selection import GridSearchCV


In [44]:
from dask_ml.model_selection import train_test_split

In [2]:
from dask_ml.metrics import mean_squared_error, r2_score
import dask.array as da
from distributed import default_client
import dask.dataframe as dd
from dask_ml.model_selection import train_test_split
from dask_ml.linear_model import LinearRegression
from dask_ml.xgboost import XGBRegressor
from dask_ml.model_selection import GridSearchCV
from dask.distributed import Client


/Users/rohithreddykota/miniconda3/lib/python3.11/site-packages/dask_xgboost/__init__.py:7: UserWarning: Dask-XGBoost has been deprecated and is no longer maintained. The functionality of this project has been included directly in XGBoost. To use Dask and XGBoost together, please use ``xgboost.dask`` instead https://xgboost.readthedocs.io/en/latest/tutorials/dask.html.
  warnings.warn(


In [2]:
# Close the existing client
client.close()

# If you also want to shut down the scheduler and all workers explicitly
# default_client().cluster.close()

NameError: name 'client' is not defined

In [3]:

# Configure the new number of workers and threads per worker
n_workers = 4  # For example, changing to 8 workers
threads_per_worker = 2  # And adjusting threads per worker

# Start a new client with the updated configuration
client = Client(n_workers=n_workers, threads_per_worker=threads_per_worker)
print("Dashboard link:", client.dashboard_link)


Dashboard link: http://127.0.0.1:8787/status


In [4]:
train = dd.read_csv("train_FE.csv", blocksize='64MB')
train = train.drop("Unnamed: 0", axis=1)


In [5]:
X = train[train.columns[3:]]
X = X.drop(["pickup_day_of_week", "euc_distance"], axis=1)
y = train['fare_amount'].to_frame()  # Assuming 'fare_amount' is your target column

# Convert to dask arrays if necessary for some models
X, y = X.to_dask_array(lengths=True), y.to_dask_array(lengths=True)


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)


In [8]:

lr = LinearRegression()
lr.fit(X_train, y_train)


LinearRegression()

In [20]:
lr_y_pred = lr.predict(X_test)
mse = mean_squared_error(y_test, lr_y_pred)
rmse = da.sqrt(mse).compute()  # Compute RMSE from MSE
r2 = r2_score(y_test, lr_y_pred).compute()  # Compute R²

print(f"RMSE: {rmse}")
print(f"R² score: {r2}")


In [15]:
import joblib
from sklearn.model_selection import cross_val_score
with joblib.parallel_backend('dask'):
    scores = cross_val_score(lr, X_train, y_train, cv=5)

print(f"Cross-validation scores: {scores}")
print(f"Average score: {scores.mean()}")


Cross-validation scores: [0.74191947 0.75792071 0.74513824 0.77729981 0.76624039]
Average score: 0.757703724491168


In [16]:
from dask_ml.metrics import mean_squared_error, r2_score
import dask.array as da
from distributed import default_client
import dask.dataframe as dd
from dask_ml.model_selection import train_test_split
from dask_ml.linear_model import LinearRegression
from dask_ml.xgboost import XGBRegressor
# from dask_ml.ensemble import RandomForestRegressor
from dask_ml.model_selection import GridSearchCV
from dask.distributed import Client
import joblib
from sklearn.model_selection import cross_val_score


# Configure the new number of workers and threads per worker
n_workers = 4  # For example, changing to 8 workers
threads_per_worker = 2  # And adjusting threads per worker

# Start a new client with the updated configuration
client = Client(n_workers=n_workers, threads_per_worker=threads_per_worker)
print("Dashboard link:", client.dashboard_link)
train = dd.read_csv("train_FE.csv", blocksize='64MB')
train = train.drop("Unnamed: 0", axis=1)

X = train[train.columns[3:]]
X = X.drop(["pickup_day_of_week", "euc_distance"], axis=1)
y = train['fare_amount'].to_frame()  # Assuming 'fare_amount' is your target column

# Convert to dask arrays if necessary for some models
X, y = X.to_dask_array(lengths=True), y.to_dask_array(lengths=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

lr = LinearRegression()
lr.fit(X_train, y_train)

lr_y_pred = lr.predict(X_test)
mse = mean_squared_error(y_test, lr_y_pred)
rmse = da.sqrt(mse).compute()  # Compute RMSE from MSE
r2 = r2_score(y_test, lr_y_pred).compute()  # Compute R²

print(f"RMSE: {rmse}")
print(f"R² score: {r2}")


with joblib.parallel_backend('dask'):
    scores = cross_val_score(lr, X_train, y_train, cv=5)

print(f"Cross-validation scores: {scores}")
print(f"Average score: {scores.mean()}")


/Users/rohithreddykota/miniconda3/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 49711 instead
  warnings.warn(


Dashboard link: http://127.0.0.1:49711/status


In [7]:
import numpy as np
import dask.array as da
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import mean_squared_error, r2_score
from dask.distributed import Client
import joblib

dt = DecisionTreeRegressor()

# Since DecisionTreeRegressor does not natively support Dask, 
# the fitting process here won't be parallelized beyond what sklearn internally supports (e.g., through joblib for some operations)
dt.fit(X_train.compute(), y_train.compute())

# Parallelize cross-validation using Dask with Joblib
with joblib.parallel_backend('dask'):
    scores = cross_val_score(dt, X_train.compute(), y_train.compute(), cv=6)
    # cross_val_predict is not demonstrated to be parallelized directly via Dask as its primary goal isn't scoring or hyperparameter tuning
    dt_y_pred = cross_val_predict(dt, X_test.compute(), y_test.compute(), cv=6)

# Compute metrics
rmse_dt = np.sqrt(mean_squared_error(y_test.compute(), dt_y_pred))
r2_dt = r2_score(y_test.compute(), dt_y_pred)

print(f"RMSE: {rmse_dt}")
print(f"Mean squared error: {mean_squared_error(y_test.compute(), dt_y_pred)}")
print(f"Variance score: {r2_dt}")

RMSE: 5.740503063135383
Mean squared error: 32.95337541786671
Variance score: 0.6354104415100583


In [8]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import joblib
import numpy as np

rf = RandomForestRegressor()

with joblib.parallel_backend('dask'):
    rf.fit(X_train.compute(), y_train.compute())  # `.compute()` ensures data is in memory. Be mindful of large datasets.

# Predict in a parallel computation context
with joblib.parallel_backend('dask'):
    rf_y_pred = rf.predict(X_test.compute())  # Ensure X_test is computed if it's a Dask object.

# Calculate metrics
rmse_rf = np.sqrt(mean_squared_error(y_test.compute(), rf_y_pred))  # Ensure y_test is computed if it's a Dask object.
r2_rf = r2_score(y_test.compute(), rf_y_pred)

print(f"RMSE: {rmse_rf}")
print(f"Mean squared error: {mean_squared_error(y_test.compute(), rf_y_pred)}")
print(f"Variance score: {r2_rf}")

/Users/rohithreddykota/miniconda3/lib/python3.11/site-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RMSE: 3.8566924153128532
Mean squared error: 14.874076386331689
Variance score: 0.8354361920782742


In [38]:
client.close()


In [39]:

client = Client(n_workers=4, threads_per_worker=1)

In [ ]:
import dask.array as da
from dask.distributed import Client
import dask_xgboost as dxgb
import xgboost as xgb
from dask_ml.model_selection import GridSearchCV, train_test_split
from dask_ml.metrics import mean_squared_error
from dask_ml.wrappers import ParallelPostFit
import numpy as np

xgbr = dxgb.XGBRegressor()

# Convert to Dask Array if your data is not already in this format
# X_train, y_train = da.rechunk(X_train, chunks=(1000, X_train.shape[1])), da.from_array(y_train, chunks=1000)

# Train model
xgbr.fit(X_train, y_train)

# Predictions
xgbr_y_pred = xgbr.predict(X_test)


params = {
    "max_depth": [7],
    # "eta": [0.2],
    "objective": ["reg:squarederror"],
    "eval_metric": ["rmse"],
    "learning_rate": [0.5],
}

# Note: dask_ml's GridSearchCV does not require the `n_jobs` parameter as parallelism is managed by Dask
grid_search = GridSearchCV(estimator=xgbr, param_grid=params, cv=5)

grid_search.fit(X_train, y_train_dask)

xgbr2_y_pred = grid_search.predict(X_test.compute())

# Ensure y_test is a NumPy array for comparison
y_test_computed = y_test.compute()

rmse_xgbr2 = np.sqrt(mean_squared_error(y_test_computed, xgbr2_y_pred))
print(f"RMSE: {rmse_xgbr2}")
print(f"Mean squared error: {mean_squared_error(y_test_computed, xgbr2_y_pred)}")
print(f"Variance score: {r2_score(y_test_computed, xgbr2_y_pred)}")


In [86]:
client.close()


In [87]:
from dask.distributed import Client
client = Client(n_workers=8, threads_per_worker=1)


In [88]:

chunks = 1000

In [89]:
print("Dashboard link:", client.dashboard_link)

Dashboard link: http://127.0.0.1:8787/status


In [90]:
import dask.array as da
from dask.distributed import Client
import dask_xgboost as dxgb
import xgboost as xgb
from dask_ml.model_selection import GridSearchCV, train_test_split
from dask_ml.metrics import mean_squared_error
from dask_ml.wrappers import ParallelPostFit
import numpy as np
from dask_ml.xgboost import XGBRegressor
from dask_ml.metrics import mean_squared_error, r2_score
import dask.array as da
from distributed import default_client
import dask.dataframe as dd
from dask_ml.model_selection import train_test_split
from dask_ml.linear_model import LinearRegression
from dask_ml.xgboost import XGBRegressor

# from dask_ml.ensemble import RandomForestRegressor
from dask_ml.model_selection import GridSearchCV
from dask.distributed import Client
import joblib
from sklearn.model_selection import cross_val_score
import dask.array as da
from dask.distributed import Client
import dask_xgboost as dxgb
import xgboost as xgb
from dask_ml.model_selection import GridSearchCV, train_test_split
from dask_ml.metrics import mean_squared_error
from dask_ml.wrappers import ParallelPostFit
import numpy as np
import numpy as np
import dask.array as da
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import mean_squared_error, r2_score
from dask.distributed import Client
import joblib


In [91]:
train = dd.read_csv("train_FE.csv", blocksize="64MB")
train = train.drop("Unnamed: 0", axis=1)

X = train[train.columns[3:]]
X = X.drop(["pickup_day_of_week", "euc_distance"], axis=1)
y = train["fare_amount"].to_frame()  # Assuming 'fare_amount' is your target column

# Convert to dask arrays if necessary for some models
X, y = X.to_dask_array(lengths=True), y.to_dask_array(lengths=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:

xgbr = XGBRegressor()

# Convert to Dask Array if your data is not already in this format
# X_train, y_train = da.rechunk(X_train, chunks=(1000, X_train.shape[1])), da.from_array(y_train, chunks=1000)

# Train model
xgbr.fit(X_train, y_train)

# Predictions
xgbr_y_pred = xgbr.predict(X_test)


params = {
    "max_depth": [7],
    "objective": ["reg:squarederror"],
    "eval_metric": ["rmse"],
    "learning_rate": [1],
}

grid_search = GridSearchCV(estimator=xgbr, param_grid=params, cv=4)

# rechunking the data
X_train = da.rechunk(X_train, chunks=(chunks, X_train.shape[1]))
y_train = da.rechunk(y_train, chunks=chunks)


In [ ]:

grid_search.fit(X_train, y_train)


In [ ]:

xgbr2_y_pred = grid_search.predict(X_test.compute())


In [ ]:

# Ensure y_test is a NumPy array for comparison
y_test_computed = y_test.compute()

rmse_xgbr2 = np.sqrt(mean_squared_error(y_test_computed, xgbr2_y_pred))
print(f"RMSE: {rmse_xgbr2}")
print(f"Mean squared error: {mean_squared_error(y_test_computed, xgbr2_y_pred)}")
print(f"Variance score: {r2_score(y_test_computed, xgbr2_y_pred)}")